# 誤差逆伝播法

## 計算グラフ
計算の過程をグラフによって表したもの

- **順伝播**: 計算グラフにおいて計算を左から右へ進めること
- **逆伝播**: 計算グラフにおいて計算を右から左へ進めること

## 連鎖律
ある関数が合成関数で表される場合，その合成関数の微分は，合成関数を構成するそれぞれの関数の微分の積によって表すことができる．

## 逆伝播

## 単純なレイヤの実装
計算グラフの乗算ノードを「乗算レイヤ」，加算ノードを「加算レイヤ」と呼ぶ．

### ch05/layer_naive.py

In [1]:
class MulLayer:
    """
    乗算レイヤ
    """
    def __init__(self):
        self.x = None
        self.y = None

    def forward(self, x, y):
        """
        順伝播
        入力した2つの値を乗算して出力
        """
        self.x = x
        self.y = y                
        out = x * y

        return out

    def backward(self, dout):
        """
        逆伝播
        入力した微分に対して、逆伝播された値を乗算して出力
        """
        dx = dout * self.y
        dy = dout * self.x

        return dx, dy


class AddLayer:
    """
    加算レイヤ
    """
    def __init__(self):
        pass # 何も行わない

    def forward(self, x, y):
        """
        順伝播
        入力した2つの値を加算して出力
        """
        out = x + y

        return out

    def backward(self, dout):
        """
        逆伝播
        入力した微分に対して、逆伝播された値を加算して出力
        """
        dx = dout * 1
        dy = dout * 1

        return dx, dy

### ch05/buy_apple.py

In [2]:
apple = 100
apple_num = 2
tax = 1.1

mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

# forward
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

# backward
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print("price:", int(price))
print("dApple:", dapple)
print("dApple_num:", int(dapple_num))
print("dTax:", dtax)

price: 220
dApple: 2.2
dApple_num: 110
dTax: 200


### ch05/buy_apple_orange.py

In [3]:
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

# layer
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

# forward
apple_price = mul_apple_layer.forward(apple, apple_num)  # (1)
orange_price = mul_orange_layer.forward(orange, orange_num)  # (2)
all_price = add_apple_orange_layer.forward(apple_price, orange_price)  # (3)
price = mul_tax_layer.forward(all_price, tax)  # (4)

# backward
dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice)  # (4)
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price)  # (3)
dorange, dorange_num = mul_orange_layer.backward(dorange_price)  # (2)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)  # (1)

print("price:", int(price))
print("dApple:", dapple)
print("dApple_num:", int(dapple_num))
print("dOrange:", dorange)
print("dOrange_num:", int(dorange_num))
print("dTax:", dtax)


price: 715
dApple: 2.2
dApple_num: 110
dOrange: 3.3000000000000003
dOrange_num: 165
dTax: 650


## 活性化関数レイヤの実装

### ch05/two_layer_net.py

In [2]:
import sys, os
sys.path.append(os.pardir)  # 親ディレクトリのファイルをインポートするための設定
import numpy as np
from common.layers import *
from common.gradient import numerical_gradient
from collections import OrderedDict



class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size, weight_init_std = 0.01):
        """
        初期化を行う．
        
        Args:
            input_size:入力層のニューロンの数
            hidden_size:隠れ層のニューロンの数
            output_size:出力層のニューロンの数
            weight_init_std:重みの標準偏差
        """
        # 重みの初期化
        self.params = {} # パラメータを保持するディクショナリ変数
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size) 
        self.params['b2'] = np.zeros(output_size)

        # レイヤの生成
        # 順番付きのディクショナリにすることで，forward()メソッドを呼び出すだけで処理が完了する
        self.layers = OrderedDict() # レイヤを保持する順番付きディクショナリ変数
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])

        self.lastLayer = SoftmaxWithLoss() # 最後のレイヤ
        
    def predict(self, x):
        """
        推論を行う
        
        Args:
            x:入力データ
        """
        for layer in self.layers.values():
            x = layer.forward(x)
        
        return x
        
    def loss(self, x, t):
        """
        損失関数を求める
        
        Args:
            x:入力データ
            t:教師データ
        """
        y = self.predict(x)
        return self.lastLayer.forward(y, t)
    
    def accuracy(self, x, t):
        """
        認識精度を求める
        
        Args:
            x:入力データ
            t:教師データ
        """
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1 : t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
        
    def numerical_gradient(self, x, t):
        """
        勾配を求める（数値微分）
        
        Args:
            x:入力データ
            t:教師データ
        """
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
        return grads
        
    def gradient(self, x, t):
        """
        勾配を求める（誤差逆伝播法）
        
        Args:
            x:入力データ
            t:教師データ
        """
        # forward
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.lastLayer.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        # 設定
        grads = {}
        grads['W1'], grads['b1'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W2'], grads['b2'] = self.layers['Affine2'].dW, self.layers['Affine2'].db

        return grads

### 誤差逆伝播法の勾配確認
- **勾配確認**: 数値微分で勾配を求めた結果と，誤差逆伝播法で求めた勾配の結果が一致することを確認すること
### ch05/gradient_check.py

In [5]:
import sys, os
sys.path.append(os.pardir)  # 親ディレクトリのファイルをインポートするための設定
import numpy as np
from dataset.mnist import load_mnist

# データの読み込み
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

x_batch = x_train[:3]
t_batch = t_train[:3]

grad_numerical = network.numerical_gradient(x_batch, t_batch)
grad_backprop = network.gradient(x_batch, t_batch)

for key in grad_numerical.keys():
    diff = np.average( np.abs(grad_backprop[key] - grad_numerical[key]) )
    print(key + ":" + str(diff))

W1:1.7966944727947356e-06
b1:2.3788128390528938e-05
W2:6.2129618899126245e-09
b2:1.3941309729426087e-07


### 誤差逆伝播法を使った学習

### ch05/train_neuralnet.py

In [6]:
import sys, os
sys.path.append(os.pardir)

import numpy as np
from dataset.mnist import load_mnist

# データの読み込み
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    # 誤差逆伝播法によって勾配を求める
    #grad = network.numerical_gradient(x_batch, t_batch)
    grad = network.gradient(x_batch, t_batch)
    
    # 更新
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
    
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print(train_acc, test_acc)


0.10748333333333333 0.1064
0.90525 0.9047
0.9235833333333333 0.9245
0.9364666666666667 0.9352
0.9448166666666666 0.9436
0.9519166666666666 0.9513
0.95735 0.9543
0.9620333333333333 0.959
0.9645833333333333 0.9606
0.9675 0.9619
0.9677 0.9629
0.9703833333333334 0.9637
0.9718833333333333 0.9661
0.9733333333333334 0.9664
0.9764 0.9697
0.9770666666666666 0.9694
0.9778666666666667 0.969
